In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
gap_train= pd.read_pickle('./temp_result/train_kaggle_processed_PPA_PCA_PPA')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed_PPA_PCA_PPA')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed_PPA_PCA_PPA')

In [3]:
NUM_TRAIN = gap_train.count().values[0]
NUM_TEST = gap_test.count().values[0]
NUM_VALID = gap_valid.count().values[0]
def label(A,B):
    if A is True:
        return 0
    if B is True:
        return 1
    return 2
def switch_label(l):
    if l==2:
        return 2
    return 1-l

In [4]:
def process_prediction(pred):
    s = pred.shape[0]//2
    pred0 = pred[0:s,:]
    pred1 = pred[s:,:]
    pred1 = pred1[:,[1,0,2]]
    pred_out = pred0+pred1
    return pred_out/2

In [5]:
def switch_A_B(df):
    columnsTitles = ["B_dist","A_dist","B_pos", "A_pos","pron_pos", "B_vector", "A_vector","pron_vector","product_vector_B","product_vector_A","label"]
    df2=df.reindex(columns=columnsTitles).copy()
    df2.columns = df.columns
    df2.label = df2.label.map(switch_label)
    return pd.concat([df,df2],axis = 0, sort = False)

In [6]:
import torch
def compute_loss(sub_df,test_data):
    pred = torch.Tensor(np.log(sub_df.loc[:,['A','B','NEITHER']].values))
    label = torch.LongTensor(list(test_data.label))
    loss = torch.nn.NLLLoss()
    loss_value = loss(pred,label).item()
    return loss_value

In [7]:
train_data = gap_train.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
train_data.A_vector = train_data.A_vector.map(lambda x:np.mean(x,axis = 0))
train_data.B_vector = train_data.B_vector.map(lambda x:np.mean(x,axis = 0))
train_data.pron_vector = train_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
train_data["product_vector_A"] = train_data.A_vector*train_data.pron_vector
train_data["product_vector_B"] = train_data.B_vector*train_data.pron_vector
train_data["label"] = train_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
train_data = train_data.drop(columns= ["A-coref","B-coref"])
#train_data = switch_A_B(train_data)

In [8]:
test_data = gap_test.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
test_data.A_vector = test_data.A_vector.map(lambda x:np.mean(x,axis = 0))
test_data.B_vector = test_data.B_vector.map(lambda x:np.mean(x,axis = 0))
test_data.pron_vector = test_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
test_data["product_vector_A"] = test_data.A_vector*test_data.pron_vector
test_data["product_vector_B"] = test_data.B_vector*test_data.pron_vector
test_data["label"] = test_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
test_data = test_data.drop(columns= ["A-coref","B-coref"])
#test_data = switch_A_B(test_data)

In [9]:
valid_data = gap_valid.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
valid_data.A_vector = valid_data.A_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.B_vector = valid_data.B_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.pron_vector = valid_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
valid_data["product_vector_A"] = valid_data.A_vector*valid_data.pron_vector
valid_data["product_vector_B"] = valid_data.B_vector*valid_data.pron_vector
valid_data["label"] = valid_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
valid_data = valid_data.drop(columns= ["A-coref","B-coref"])
#valid_data = switch_A_B(valid_data)

In [10]:
columns = train_data.columns[:-1]
X_train = np.concatenate([np.array(list(train_data[col])).reshape(train_data.shape[0],-1) for col in columns],axis = 1)
y_train = list(train_data.label)   

In [11]:
X_valid = np.concatenate([np.array(list(valid_data[col])).reshape(valid_data.shape[0],-1) for col in columns],axis = 1)
y_valid = list(valid_data.label)   

In [12]:
X_test = np.concatenate([np.array(list(test_data[col])).reshape(test_data.shape[0],-1) for col in columns],axis = 1)
y_test= list(test_data.label)   

In [13]:
import pickle
f = open( "./temp_result/base_model_data_part4", "wb" )
pickle.dump(X_train,  f)
pickle.dump(y_train,  f)
pickle.dump(X_valid,  f)
pickle.dump(y_valid,  f)
pickle.dump(X_test,  f)
pickle.dump(y_test,  f)
f.close()

In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial',C = 0.02).fit(X_train, y_train)

/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [15]:
pred_lr = lr.predict_proba(X_test)

In [16]:
#pred_lr = process_prediction(pred_lr)
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_lr


sub_df.to_csv("./test_and_submit/submission+model+lr@"+str(datetime.datetime.now())+".csv", index=False)

sub_df.head()


,ID,A,B,NEITHER
0,development-1,0.671711,0.222999,0.105290
1,development-2,0.814224,0.127163,0.058613
2,development-3,0.074179,0.904455,0.021366
3,development-4,0.044189,0.806505,0.149307
4,development-5,0.012728,0.913857,0.073415


In [17]:
compute_loss(sub_df,test_data)

0.5332767963409424

In [18]:
pred_lr_train = lr.predict_proba(X_train)

In [19]:
import pickle
f = open( "./temp_result/lr_result_base", "wb" )
pickle.dump(pred_lr_train,  f)
pickle.dump(pred_lr,  f)
f.close()

In [20]:
from sklearn.svm import SVC
svm = SVC(C = 7.0,verbose=True,probability = True,gamma = "auto",class_weight='balanced').fit(X_train, y_train)

[LibSVM]

In [21]:
pred_svm = svm.predict_proba(X_test)

In [22]:
#pred_svm = process_prediction(pred_svm)
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_svm

In [23]:
sub_df.to_csv("./test_and_submit/submission+model+svm@"+str(datetime.datetime.now())+".csv", index=False)
sub_df.head()

,ID,A,B,NEITHER
0,development-1,0.757078,0.194732,0.048190
1,development-2,0.842697,0.106756,0.050547
2,development-3,0.052195,0.914334,0.033471
3,development-4,0.039774,0.779345,0.180880
4,development-5,0.015871,0.880888,0.103242


In [24]:
compute_loss(sub_df,test_data)

0.5223201513290405

In [25]:
pred_svm_train = svm.predict_proba(X_train)

In [26]:
import pickle
f = open( "./temp_result/svm_result_base", "wb" )
pickle.dump(pred_svm_train,  f)
pickle.dump(pred_svm,  f)
f.close()

In [27]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(256*5+5, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.95),
            nn.Linear(128, 3)
        )
        nn.init.xavier_uniform_(self.layers[0].weight)
        nn.init.xavier_uniform_(self.layers[-1].weight)
    def forward(self, x):
        # convert tensor (128, 1, 28, 28) --> (128, 1*28*28)
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

In [28]:
EPOCHS = 500
batch_size = 25
mlp = MLP()
mlp.cuda()
opt = torch.optim.Adam(mlp.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()#weight = torch.Tensor([1.0,1.0,5.0]))
loss_fn.cuda()

CrossEntropyLoss()

In [29]:
for e in range(EPOCHS):
    for b in range(0,X_train.shape[0],batch_size):
        mlp.train()
        batch_data = X_train[b:b+batch_size,:]
        batch_label = y_train[b:b+batch_size]
        output = mlp(torch.Tensor(batch_data).cuda())
        batch_label = torch.LongTensor(batch_label).cuda()
        loss = loss_fn(output,batch_label)
        l2_norm = torch.norm(mlp.layers[-1].weight, p=2)
        loss += l2_norm*0.09
        l2_norm = torch.norm(mlp.layers[0].weight, p=2)
        loss += l2_norm*0.03
        opt.zero_grad()
        loss.backward()
        opt.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(e + 1, EPOCHS, loss.item()))

epoch [1/500], loss:1.6541
epoch [2/500], loss:1.4735
epoch [3/500], loss:1.2074
epoch [4/500], loss:1.4123
epoch [5/500], loss:1.1045
epoch [6/500], loss:1.1921
epoch [7/500], loss:1.2512
epoch [8/500], loss:1.1826
epoch [9/500], loss:1.0633
epoch [10/500], loss:1.0974
epoch [11/500], loss:0.8932
epoch [12/500], loss:1.1556
epoch [13/500], loss:1.0389
epoch [14/500], loss:1.1528
epoch [15/500], loss:0.9368
epoch [16/500], loss:0.9326
epoch [17/500], loss:0.9155
epoch [18/500], loss:0.9478
epoch [19/500], loss:0.8830
epoch [20/500], loss:0.8098
epoch [21/500], loss:0.8873
epoch [22/500], loss:0.8797
epoch [23/500], loss:0.9775
epoch [24/500], loss:0.8956
epoch [25/500], loss:0.9052
epoch [26/500], loss:0.9601
epoch [27/500], loss:0.9519
epoch [28/500], loss:0.8506
epoch [29/500], loss:0.7798
epoch [30/500], loss:0.9190
epoch [31/500], loss:0.9266
epoch [32/500], loss:0.8708
epoch [33/500], loss:0.9006
epoch [34/500], loss:0.8527
epoch [35/500], loss:0.8534
epoch [36/500], loss:0.7567
e

epoch [289/500], loss:0.8561
epoch [290/500], loss:0.9598
epoch [291/500], loss:0.8869
epoch [292/500], loss:0.9067
epoch [293/500], loss:0.9259
epoch [294/500], loss:0.8195
epoch [295/500], loss:0.8315
epoch [296/500], loss:0.8078
epoch [297/500], loss:0.9228
epoch [298/500], loss:0.8561
epoch [299/500], loss:0.9028
epoch [300/500], loss:0.9378
epoch [301/500], loss:0.9608
epoch [302/500], loss:0.8112
epoch [303/500], loss:0.8854
epoch [304/500], loss:0.7879
epoch [305/500], loss:0.9194
epoch [306/500], loss:0.8478
epoch [307/500], loss:0.7865
epoch [308/500], loss:0.9302
epoch [309/500], loss:0.8079
epoch [310/500], loss:0.7579
epoch [311/500], loss:0.8085
epoch [312/500], loss:1.0917
epoch [313/500], loss:0.8174
epoch [314/500], loss:0.8863
epoch [315/500], loss:0.8572
epoch [316/500], loss:0.7580
epoch [317/500], loss:0.9704
epoch [318/500], loss:0.6858
epoch [319/500], loss:0.9352
epoch [320/500], loss:0.8349
epoch [321/500], loss:0.7662
epoch [322/500], loss:0.8531
epoch [323/500

In [30]:
mlp.eval()
pred_mlp = torch.nn.Softmax(dim = 1)(mlp(torch.Tensor(X_test[:,:]).cuda())).cpu().data.numpy()
pred_mlp_train = torch.nn.Softmax(dim = 1)(mlp(torch.Tensor(X_train[:,:]).cuda())).cpu().data.numpy()

In [31]:
#pred_mlp = process_prediction(pred_mlp)

In [32]:
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_mlp
sub_df.to_csv("./test_and_submit/submission+model+mlp@"+str(datetime.datetime.now())+".csv", index=False)
sub_df.head()

,ID,A,B,NEITHER
0,development-1,0.484329,0.375021,0.140650
1,development-2,0.961475,0.023558,0.014966
2,development-3,0.042973,0.928943,0.028083
3,development-4,0.080485,0.752520,0.166994
4,development-5,0.004847,0.987228,0.007925


In [33]:
compute_loss(sub_df,test_data)

0.5496715307235718

In [34]:
import pickle
f = open( "./temp_result/mlp_result_base", "wb" )
pickle.dump(pred_mlp_train,  f)
pickle.dump(pred_mlp,  f)
f.close()

In [35]:
import pickle
f = open( "./temp_result/label", "wb" )
pickle.dump(y_train,  f)
pickle.dump(y_test,  f)
f.close()